In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pygame


NOTES = ["DO", "RE", "MI", "FA", "SOL", "LA", "SI", "DO"]
SOUNDS = ["sounds/do.mp3", "sounds/re.mp3", "sounds/mi.mp3",
          "sounds/fa.mp3", "sounds/sol.mp3", "sounds/la.mp3",
          "sounds/si.mp3", "sounds/do.mp3"]

pygame.mixer.init()
note_sounds = [pygame.mixer.Sound(s) for s in SOUNDS]

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

last_piano = None
alpha = 0.88
last_played = -1 


while True:
    ret, frame = cap.read()
    if not ret:
        break

    h_img, w_img, _ = frame.shape

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)

    edges = cv2.Canny(blur, 20, 120)

    kernel = np.ones((5, 5), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=2)
    edges = cv2.erode(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    piano_rect = None
    max_area = 0
    frame_area = h_img * w_img

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = cv2.contourArea(cnt)

        if area > max_area and area > frame_area * 0.12 and w > h:
            max_area = area
            piano_rect = (x, y, w, h)

    if piano_rect is not None:
        if last_piano is None:
            last_piano = piano_rect
        else:
            x = int(alpha * last_piano[0] + (1 - alpha) * piano_rect[0])
            y = int(alpha * last_piano[1] + (1 - alpha) * piano_rect[1])
            w = int(alpha * last_piano[2] + (1 - alpha) * piano_rect[2])
            h = int(alpha * last_piano[3] + (1 - alpha) * piano_rect[3])
            last_piano = (x, y, w, h)

    piano_rect = last_piano
    key_pressed = -1


    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        hand_landmarks = result.multi_hand_landmarks[0]
        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        index_finger = hand_landmarks.landmark[8]
        fx = int(index_finger.x * w_img)
        fy = int(index_finger.y * h_img)

        cv2.circle(frame, (fx, fy), 10, (0, 255, 255), -1)

    else:
        fx, fy = None, None


    if piano_rect:
        x, y, w, h = piano_rect
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        key_width = w // 8

        for i in range(8):
            kx = x + i * key_width
            ky = y

            pressed = False
            if fx and fy:
                if kx < fx < kx + key_width and ky < fy < ky + h:
                    pressed = True
                    key_pressed = i

            color = (0, 255, 255) if pressed else (255, 0, 0)

            cv2.rectangle(frame, (kx, y), (kx + key_width, y + h), color, 2)

            cv2.putText(frame, NOTES[i], (kx + 10, y + h - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        if key_pressed != -1 and key_pressed != last_played:
            note_sounds[key_pressed].play()
            last_played = key_pressed

        if key_pressed == -1:
            last_played = -1

    cv2.imshow("Piano Virtual", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break


cap.release()
cv2.destroyAllWindows()
pygame.quit()


c:\Users\Usuario\anaconda3\envs\VC_Final\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
